In [5]:
import torch
from datasets import load_dataset


#定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        dataset = load_dataset(path='./ChnSentiCorp', split=split)

        def f(data):
            return len(data['text']) > 30

        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']

        return text


dataset = Dataset('train')

len(dataset), dataset[0]

(9192,
 '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般')

In [12]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('./model_dir/bert-base-chinese')

token

BertTokenizer(name_or_path='./model_dir/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
def collate_fn(data):
    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    #把第15个词固定替换为mask
    labels = input_ids[:, 15].reshape(-1).clone()
    input_ids[:, 15] = token.get_vocab()[token.mask_token]

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
print(token.decode(input_ids[0]))
print(token.decode(labels[0]))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape

574
[CLS] 外 观 漂 亮 ， 最 大 的 卖 点 。 键 盘 大 [MASK] 联 想 ｓ１０ 被 淘 汰 的 原 因 。 电 池 [SEP]
，


(torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16]))

In [13]:
from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('./model_dir/bert-base-chinese')

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

out.last_hidden_state.shape

torch.Size([16, 30, 768])

In [9]:
#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768, token.vocab_size, bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        out = self.decoder(out.last_hidden_state[:, 15])

        return out


model = Model()

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

torch.Size([16, 21128])

In [10]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(5):
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader):
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)

        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 50 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            print(epoch, i, loss.item(), accuracy)

/home/user_home/xiaomingxu/mambaforge/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0 10.092287063598633 0.0
0 50 8.042887687683105 0.125
0 100 7.124838829040527 0.0625
0 150 5.69180965423584 0.125
0 200 4.263329029083252 0.5625
0 250 4.823337554931641 0.125
0 300 4.204056739807129 0.3125
0 350 4.238401412963867 0.3125
0 400 4.561729431152344 0.4375
0 450 4.504779815673828 0.3125
0 500 2.0661520957946777 0.75
0 550 2.909170627593994 0.5
1 0 2.9464640617370605 0.625
1 50 3.2296972274780273 0.4375
1 100 2.951188802719116 0.4375
1 150 3.360457181930542 0.375
1 200 1.510347604751587 0.75
1 250 1.9105085134506226 0.75
1 300 1.6942493915557861 0.6875
1 350 2.536846399307251 0.4375
1 400 1.6170709133148193 0.6875
1 450 2.510134696960449 0.5625
1 500 2.320904016494751 0.5
1 550 0.9959594011306763 0.8125
2 0 1.072443962097168 0.75
2 50 1.260981798171997 0.75
2 100 1.3583310842514038 0.75
2 150 0.9122206568717957 0.75
2 200 0.680137038230896 0.875
2 250 0.6329259276390076 0.9375
2 300 1.198613166809082 0.75
2 350 0.6955475807189941 0.8125
2 400 0.5126619935035706 0.9375
2 450

In [11]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 15:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

        print(token.decode(input_ids[0]))
        print(token.decode(labels[0]), token.decode(labels[0]))

    print(correct / total)


test()

Filter:   0%|          | 0/1200 [00:00<?, ? examples/s]

0
[CLS] 孩 子 已 经 看 过 一 遍 了 ， 到 现 在 回 [MASK] 只 要 看 到 托 马 斯 还 是 拿 起 来 读 [SEP]
家 家
1
[CLS] 容 易 产 生 指 纹 。 不 习 惯 分 区 。 由 [MASK] 出 货 量 大 了 ， 我 觉 得 在 配 货 的 [SEP]
于 于
2
[CLS] 超 差 ！ ！ ！ 建 议 所 有 人 不 要 来 住 [MASK] 大 门 开 在 胡 同 里 ， 找 不 着 。 每 [SEP]
！ ！
3
[CLS] 因 为 前 些 时 间 对 心 理 学 感 兴 趣 ， [MASK] 了 上 下 集 。 好 久 没 看 小 说 了 ， [SEP]
买 买
4
[CLS] vista 系 统 跑 起 来 慢 ， 准 备 干 掉 装 [UNK] [MASK] 没 有 摄 像 头 、 读 卡 器 ， 买 的 时 [SEP]
。 。
5
[CLS] 刚 拿 回 来 发 现 指 示 灯 位 置 开 胶 了 [MASK] 直 接 到 联 想 客 服 用 双 面 胶 粘 上 [SEP]
， ，
6
[CLS] 用 起 来 还 不 错 ， 本 人 还 有 几 张 2000 [MASK] 100 1000 - 50 东 券 要 的 加 qq 673946 [SEP]
- -
7
[CLS] 之 前 买 过 一 本 [UNK] 版 [UNK] 三 联 买 的 109 [MASK] 像 太 贵 了 啦 ！ 那 时 太 想 买 了 现 [SEP]
好 好
8
[CLS] 贝 贝 熊 1 - 30 我 和 女 儿 都 很 喜 欢 [MASK] 我 也 通 过 它 们 间 接 的 对 孩 子 进 [SEP]
， ，
9
[CLS] 看 到 推 荐 的 很 好 ， 就 买 了 。 宝 宝 [MASK] 像 不 喜 欢 ， 连 看 也 不 看 就 扔 到 [SEP]
好 好
10
[CLS] 一 年 前 我 们 给 孩 子 6 岁 的 儿 子 买 [MASK] 卡 梅 拉 的 第 一 部 ， 孩 子 非 常 喜 [SEP]
了 了
11
[CLS] 10 月 16 日 入 住 该 酒 店. 感 觉 不 错 [MASK], 离 汽 车 站 和 火 车 站 的 距 离 都 [SEP]
哦 哦
12
[CLS] [UNK] 全 兼 容